In [86]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.linear_model
import matplotlib
import pandas as pd
#from scipy.special import expit as sigmoid
%matplotlib inline

In [87]:
path = r'C:\Users\Nursultan\Documents\Data Science\breast cancer\wdbc.data.csv'
df = pd.read_csv(path, names = ['ID', 'Diagnosis',
                    'MRadius', 'MTexture', 'Mper-ter', 'Marea', 'MSmooth-es',
                    'MComp-ness', 'MConcavity', 'MConcavPoints', 'MSymm-ry',
                    'MFractDimens', 'RadiusSE', 'TextureSE', 'Per-terSE',
                    'AreaSE', 'Smooth-esSE',
                    'Comp-nessSE', 'ConcavitySE', 'ConcavPointsSE', 
                    'Symm-rySE', 'FractDimensSE', 'WRadius', 'WTexture',
                    'WPer-ter', 'WArea', 'WSmooth-es',
                    'WComp-ness', 'WConcavity', 'WConcavPoints', 
                    'WSymm-ry', 'WFractDimens'])


In [127]:
df['Malignant'] = df.Diagnosis.map({'B':0, 'M':1})
df.head()


<class 'pandas.core.frame.DataFrame'>


In [114]:
#df.index=range(0,569,1)

In [136]:
X = df.iloc[:,2:32].values

print(type(X))
#numpyMatrix = df.as_matrix()

y= df.Malignant.values
print(type(y))
#y.reshape(569,1)




<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [143]:
num_samples = len(X)
print(num_samples)
nn_inputs = 30
nn_outputs = 2
epsilon = 0.01
reg_lambda = 0.01

569


In [144]:
def calc_error(model):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    
    z1 = X.dot(W1) + b1
    a1= np.tanh(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores/np.sum(exp_scores, axis=1, keepdims = True)
    
    #calculating the error
    correct_logprobs = - np.log(probs[range(num_samples), y])
    error = np.sum(correct_logprobs)
    #optional regularization term
    error += reg_lambda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    return 1./num_samples * error

In [145]:
def feed_forward(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    #propagate forward
    #calc the dot product of input layer and its weights and add 1st bias term
    z1 = x.dot(W1) + b1
    #apply tanh activation function
    a1 = np.tanh(z1)
    #calc the dot product of the output of tanh func applied
    # to input layer and its weights and add 2nd bias term
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores/np.sum(exp_scores, axis=1, keepdims=True)
    return np.argmax(prob, axis=1)

In [149]:
#function to learn the param-s for the NN and return the model
# hid_dim - hidden layer dimension - # of nodes in hidden layer
# num_passes - num of passes through the training set for gradient descent
# print_loss - if true, prints the loss every 1000 iterations
def build_model(hid_dim, num_passes = 20000, print_loss=False):
    #initialize param-s randomly, we need to learn them
    np.random.seed(0)
    W1 = np.random.randn(nn_inputs, hid_dim)/ np.sqrt(nn_inputs)
    b1 = np.zeros((1, hid_dim))
    W2 = np.random.randn(hid_dim,nn_outputs) / np.sqrt(hid_dim)
    b2 = np.zeros((1, nn_outputs))
    
    model = {}
    
    #Gradient descent
    for i in xrange(0, num_passes):
        
        #feed forward
        z1 = X.dot(W1) + b1
        a1 = np.tanh(z1)
        z2 = a1.dot(W2) + b2
        exp_scores = np.exp(z2)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        
        #Backpropagate
        delta3 = probs
        delta3[range(num_samples), y] -= 1
        dW2 = (a1.T).dot(delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(W2.T) * (1 - np.power(a1,2))
        dW1 = np.dot(X.T, delta2)
        db1 = np.sum(delta2, axis =0)
        
        #add regularization terms (b1 and b2 aren't regularized)
        dW2 += reg_lambda * W2
        dW1 += reg_lambda * W1
        
        #Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        
        model = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        
        #optionally print the loss
        if print_loss and i % 1000 == 0:
            print "Loss after iteration %i: %f" %(i, calc_error(model))
    return model

In [154]:
# Build a model with a 10-dim hidden layer
model = build_model(3, print_loss=True)

Loss after iteration 0: 7.164825
Loss after iteration 1000: 0.903176
Loss after iteration 2000: 7.380545
Loss after iteration 3000: 2.785516
Loss after iteration 4000: 6.579739
Loss after iteration 5000: 0.670587
Loss after iteration 6000: 2.143586
Loss after iteration 7000: 6.741454
Loss after iteration 8000: 5.918293
Loss after iteration 9000: 7.922760
Loss after iteration 10000: 1.984033
Loss after iteration 11000: 0.813405
Loss after iteration 12000: 6.771853
Loss after iteration 13000: 6.903645
Loss after iteration 14000: 4.432796
Loss after iteration 15000: 7.566420
Loss after iteration 16000: 7.270446
Loss after iteration 17000: 6.384040
Loss after iteration 18000: 6.606671
Loss after iteration 19000: 0.662119
